In [1]:
import numpy as np
import cv2
import os
import shutil
from ipynb.fs.full.config import getItem_string
from ipynb.fs.full.config import getItem_int
        
def LoadImages(data):
    images = []
    labels = []
    names = []
    
    label = 0
    #read all face images from directory
    for subDirname in os.listdir(data):
        subjectPath = os.path.join(data,subDirname)
        if os.path.isdir(subjectPath):                  
            names.append(subDirname)
            for fileName in os.listdir(subjectPath):
                imgPath = os.path.join(subjectPath,fileName)
                #convert into grascale
                img = cv2.imread(imgPath,cv2.IMREAD_GRAYSCALE)
                images.append(img)
#                 print(imgPath)
#                 print(img.shape)
                labels.append(label)
            label += 1
    images = np.asarray(images)
    labels = np.asarray(labels)   
    return images,labels,names     

def FaceRec(data):  
    # create landmark detector and load lbf model:
    facemark = cv2.face.createFacemarkLBF()
    facemark.loadModel(getItem_string("opencv","facemark_model"))    
    
    X,y,names=LoadImages(data)
    #create a EigenFaceRecognizer
    model = cv2.face.EigenFaceRecognizer_create()
     #train the model
    model.train(X,y)
    print('training is done')
    #create a classifier
    face_cascade = cv2.CascadeClassifier(getItem_string("opencv","cascadeclassifier"))   
   
    #open camera   
    camera = cv2.VideoCapture(0)
    cv2.namedWindow('Camera')   
    size_of_image=getItem_int("dataset","image_size")
    while(True):
        #read a frame
        ret,frame = camera.read()
        if ret:
            #convert into grayscale
            grayscale_image = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            #detect faces
            faces = face_cascade.detectMultiScale(grayscale_image,1.3,5)   
         # (x,y)=top left , w=weight, h=height              
        for (x,y,w,h) in faces:
            #draw a rectangle contained faces
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            #create landmarks
            ok, landmarks = facemark.fit(frame, faces) 
            #crop images 
            grayscale_face = grayscale_image[y:y+h,x:x+w]          
            try:
                #crop into 250*250 
                grayscale_face = cv2.resize(grayscale_face,(size_of_image,size_of_image),interpolation=cv2.INTER_LINEAR)              
                #predict
                params = model.predict(grayscale_face)
                #print the confidence
                print('confidence: ',params[1])
                #print landmarks
                for person in landmarks:
                    for point in person[0]:
                        # circle the characteristics，total of 68
                        cv2.circle(frame, (point[0],point[1]), 5, color=(0, 255, 0))
                #print the name on the camara
                cv2.putText(frame,names[params[0]],(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,255,2)
            except:
                continue
                
            cv2.imshow('Camera',frame)            
            #press q to exit
            if cv2.waitKey(100) & 0xff == ord('q') :
                break
    camera.release()
    cv2.destroyAllWindows()
       
if __name__=='__main__':
    data = getItem_string("dataset","dataset_path")
    FaceRec(data)